# _Data Cleaning in SQL_

## 

- **<u>_Standardize Date Format_</u>**

In [ ]:
SELECT SalesDateConverted
from [Nashville Housing]

ALTER TABLE [Nashville Housing]
add SalesDateConverted Date;

UPDATE [Nashville Housing]
SET SalesDateConverted=CONVERT(date,SaleDate)

  

- _**<u>Populate Property Address data</u>**_\_

In [ ]:
SELECT a.ParcelID, a.PropertyAddress, b.ParcelID, b.PropertyAddress, ISNULL(a.PropertyAddress,b.PropertyAddress)
from [Nashville Housing]a
JOIN [Nashville Housing]b
    on a.ParcelID=b.ParcelID
    and a.[UniqueID ]<>b.[UniqueID ]
where a.PropertyAddress is NULL

UPDATE a
set propertyaddress = ISNULL(a.PropertyAddress,b.PropertyAddress)
from [Nashville Housing]a
JOIN [Nashville Housing]b
    on a.ParcelID=b.ParcelID
    and a.[UniqueID ]<>b.[UniqueID ]
where a.PropertyAddress is NULL


- **_<u>Breaking out Address into Individual Columns (Address, City, State)</u>_**

1. First way

In [ ]:
select PropertyAddress
from [Nashville Housing]

SELECT
SUBSTRING(PropertyAddress,1, CHARINDEX(',', PropertyAddress)-1) as Adress1,
    SUBSTRING(PropertyAddress,CHARINDEX(',', PropertyAddress)+1, LEN(PropertyAddress)) as Adress2

from [Nashville Housing]

ALTER TABLE [Nashville Housing]
add PropertyAddress1 nvarchar(255) ;

UPDATE [Nashville Housing]
SET PropertyAddress1 = SUBSTRING(PropertyAddress,1, CHARINDEX(',', PropertyAddress)-1) 



ALTER TABLE [Nashville Housing]
add PropertyAddress2 nvarchar(255) ;

UPDATE [Nashville Housing]
SET PropertyAddress2 =    SUBSTRING(PropertyAddress,CHARINDEX(',', PropertyAddress)+1, LEN(PropertyAddress))

SELECT *
from [Nashville Housing]


### 

1. Other way

In [ ]:
select OwnerAddress
from [Nashville Housing]
--replace comma with period
SELECT OwnerAddress,
PARSENAME(replace  (OwnerAddress,',','.'),1) as state,
PARSENAME(replace  (OwnerAddress,',','.'),2) as city,
PARSENAME(replace  (OwnerAddress,',','.'),3) as adress

from [Nashville Housing]

ALTER TABLE [Nashville Housing]
add OwnerAddressState nvarchar(255) ;

UPDATE [Nashville Housing]
SET OwnerAddressState = PARSENAME(replace  (OwnerAddress,',','.'),1) 



ALTER TABLE [Nashville Housing]
add OwnerAddressCity nvarchar(255) ;

UPDATE [Nashville Housing]
SET OwnerAddressCity =    PARSENAME(replace  (OwnerAddress,',','.'),2)


ALTER TABLE [Nashville Housing]
add OwnerAddresss nvarchar(255) ;

UPDATE [Nashville Housing]
SET OwnerAddresss =    PARSENAME(replace  (OwnerAddress,',','.'),3)

SELECT OwnerAddressCity,OwnerAddressState,OwnerAddresss
from [Nashville Housing]


## 

- **_<u>Change Y and N to Yes and No in "Sold as Vacant" field</u>_**

In [ ]:
select DISTINCT(SoldAsVacant ), COUNT (SoldAsVacant)
FROM [Nashville Housing]
GROUP BY SoldAsVacant
order by 2

SELECT SoldAsVacant ,
    case when SoldAsVacant = 'Y' then 'Yes'
         when SoldAsVacant = 'N' then 'No'
         else SoldAsVacant
         END 
FROM [Nashville Housing]

UPDATE [Nashville Housing] 
SET SoldAsVacant= case when SoldAsVacant = 'Y' then 'Yes'
                       when SoldAsVacant = 'N' then 'No'
                       else SoldAsVacant
                       END 

- **_<u>Remove Duplicates</u>_**

In [ ]:
with row_numcte as(
SELECT*,
    ROW_NUMBER() OVER(
        PARTITION BY ParcelID,
                     PropertyAddress,
                     SalePrice,
                     SaleDate,
                     LegalReference
                     order by 
                        UniqueID 
    ) row_num
FROM [Nashville Housing]
--ORDER BY ParcelID

)
delete 
from row_numcte
where row_num > 1

- **_<u>Delete Unused Columns</u>_**